In [6]:
%reset
import parcels
from parcels import ParticleFile, GeographicPolar
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt
import datetime as dati
from operator import attrgetter
from math import sqrt, pi, cos
import warnings as wr

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Coords are the same for all months

In [7]:
# For working on home computer
path = ".."

scen = np.array(["S0", "S1", "S2", "S3", "S4", "S5", "S6"])
mon = np.array(["may", "jun", "jul", "aug", "sep", "oct"])

cs = xr.open_dataset(path + "\\data\\coordinates.nc").load()
lon_grid = cs.lon_grid.values
lat_grid = cs.lat_grid.values
dep_grid = cs.dep_grid.values
cs.close()

npart = 8 # will be squared, then doubled

# Releases 30 particles at grid of lats, lons and deps every ten hours
lon_range = np.linspace(lon_grid[4,4], lon_grid[37,38], num = npart)
lat_range = np.linspace(lat_grid[4,4], lat_grid[37,38], num = npart)

# Create a grid of particles
plons = np.repeat(lon_range, npart)
plons = np.repeat(plons, 2)
plats = np.tile(lat_range, npart)
plats = np.repeat(plats, 2)
pdeps = np.repeat(4.8, repeats = npart*npart)
pdeps = np.concatenate([pdeps, np.repeat(21.5, repeats = npart*npart)], axis = 0)

def DeleteParticle(particle, fieldset, time): 
            particle.delete() # Deletes particle if it's out of bounds

#del cs, lon_range, lat_range

In [8]:
# Check they're unique!!
print(np.shape(plats), np.shape(plons), np.shape(pdeps))

x = np.empty(shape = (3,128))
for i in range(128): x[:,i] = [plats[i], plons[i], pdeps[i]]

done = np.unique(x, axis = 1)
print(np.shape(done))

#fig = plt.figure()
#ax = plt.axes()
#donplot = plt.scatter(done[1], done[0], c = done[2])
#print(done[:,0:5])
done2 = np.concatenate([done[2,4:36],done[2,4:36]])

#fig2 = plt.figure()
#ax2 = plt.axes()
#donplot2 = plt.scatter(done[1], done[0], c = done2)

plats = done[0]
plons = done[1]
pdeps = done2
print(np.shape(plats), np.shape(plons), np.shape(pdeps))

(128,) (128,) (128,)
(3, 64)
(64,) (64,) (64,)


# All in one go!

In [ ]:
scen = np.array(["S3", "S3", "S3" ])
mons = np.array(["may","sep","oct"])

for mo in range(2):
    zarrname = "D:OceanParcels\\outputs2\\" + scen[mo] + "_" + mons[mo] + ".zarr"
    dfile = "D:OceanParcels\\data\\final grids\\final_" + mons[mo] + "_" + scen[mo] + "_1.nc"
    print("Starting ", zarrname)

    ds = xr.open_dataset(dfile).load()
    times_secs = ds.times_secs.values
    U_atten = ds.U_atten.values
    V_atten = ds.V_atten.values
    W = ds.W.values
    ds.close()

    # Define fieldset
    vals = {'U': U_atten, 'V': V_atten, 'W': W}

    dims = {'U': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs},
            'V': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs},
            'W': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs}}

    fset = parcels.FieldSet.from_data(data = vals, dimensions = dims,
                      chunksize = 'auto', # allow_time_extrapolation = True,
                      gridindexingtype = 'nemo')

    del U_atten, V_atten, W, ds

    pset = parcels.ParticleSet.from_list(fieldset = fset, pclass = parcels.ScipyParticle, 
                                 lon = plons, lat = plats, depth = pdeps, 
                                 time = np.ones(npart*npart)*fset.gridset.dimrange('time')[0],
                                 repeatdt = dati.timedelta(hours = 14).total_seconds())
    pkern = pset.Kernel(parcels.AdvectionRK4_3D)
    pset.populate_indices()

    outfile = pset.ParticleFile(name = zarrname, outputdt = dati.timedelta(hours = 1))

    pset.execute(pkern, output_file = outfile, dt = dati.timedelta(minutes = 30),
                 recovery = {parcels.ErrorCode.ErrorOutOfBounds: DeleteParticle})

    del pset, fset, pkern

Starting  D:OceanParcels\outputs2\S3_may.zarr
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
INFO: Output files are stored in D:OceanParcels\outputs2\S3_may.zarr.
  0%|          | 0/2677800.0 [00:00<?, ?it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  1%|          | 18000.0/2677800.0 [00:08<20:17, 2185.15it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  1%|          | 21600.0/2677800.0 [00:16<38:24, 1152.59it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  1%|          | 25200.0/2677800.0 [00:24<54:05, 817.27it/s] Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  2%|▏         | 46800.0/2677800.0 [00:42<30:

 12%|█▏        | 309600.0/2677800.0 [08:08<1:33:56, 420.19it/s]Correct cell not found after 100000 iterations
 12%|█▏        | 313200.0/2677800.0 [08:10<2:07:06, 310.03it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏        | 313200.0/2677800.0 [08:28<2:07:06, 310.03it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏        | 316800.0/2677800.0 [08:46<3:26:59, 190.10it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏        | 316800.0/2677800.0 [08:58<3:26:59, 190.10it/s]Correct cell not found after 100000 iterations
 12%|█▏        | 320400.0/2677800.0 [09:05<3:24:48, 191.83it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 12%|█▏    

In [ ]:
#%%script echo Skipped!

import sys, glob

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)